# TensorFlow

https://www.tensorflow.org/get_started/get_started

This notebook makes learning survey throught TensorFlow.
Some part will be used in bachelor thesis.

## Tensors

Central unit of data is the **tensor**.
It is a set of primitive values shaped into an array of any
number of
dimensions.
A tensor's **rank** is its number of dimensions.

In [1]:
3.                               # rank 0, scalar
[1., 2., 3.]                     # rank 1, vector
[[1., 2., 3.], [4., 5., 6.]]     # rank 2; matrix
[[[1., 2., 3.]], [[7., 8., 9.]]] # rank 3 with shape [2, 1, 3]

[[[1.0, 2.0, 3.0]], [[7.0, 8.0, 9.0]]]

## TensorFlow Core Tutorial

In [2]:
# canonical import statement
import tensorflow as tf

### The Computational Graph

TF Core program consist of:

1. building the graph
2. running the graph

A **computational graph** is a series of TF operations
arranged into a graph of nodes.
Each node takes zero or more tensors as inputs and
produces a tensor as an output.
One type is a constant.

In [3]:
node1 = tf.constant(3.0, tf.float32)
node2 = tf.constant(4.0) # also tf.float32 implicitly
node1, node2

(<tf.Tensor 'Const:0' shape=() dtype=float32>,
 <tf.Tensor 'Const_1:0' shape=() dtype=float32>)

Noticce that the nodes does not output the values.
Instead when evaluated they would produce the values 3.0
and 4.0.

To evaluate the nodes, a **session** has to be created.
A session encapsulates the control and state of the TF
runtime.

In [4]:
sess = tf.Session()
sess.run([node1, node2])

[3.0, 4.0]

More complicated computation are built by combining nodes
with operations.

In [5]:
node3 = tf.add(node1, node2)
print('node3:', node3)
print('run:', sess.run(node3))

node3: Tensor("Add:0", shape=(), dtype=float32)
run: 7.0


A graph can be parameterized to accept external inputs
known as **placeholders**.
A **placeholder** is a promise to provide a value later.

In [6]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = a + b    # shortcut for tf.add(a, b)

This graph can be evaluated with multiple inputs by using
`feed_dict` parameter to specify tensors that provide
concrete values.

In [7]:
print(sess.run(adder_node, {a: 3, b: 4.5}))
print(sess.run(adder_node, {a: [1, 3], b: [2, 4]}))

7.5
[ 3.  7.]


In [8]:
add_and_triple = adder_node * 3.
print(sess.run(add_and_triple, {a: 3, b: 4.5}))

22.5


Machine learning wanta a model that takse arbitrary inputs.
To make the model trainable the graph need to be modifiable
to get new outputs with the same inputs.
**Variables** allow to add trainable parameters.

In [9]:
W = tf.Variable([.3], tf.float32)
b = tf.Variable([-.3], tf.float32)
x = tf.placeholder(tf.float32)
linear_model = W * x + b

Constants are initialized when `tf.constant` is called and
their value can never change.
By contrast, variables are not initialized when
`tf.Variable` is called.

In [10]:
# initialize variables
init = tf.global_variables_initializer()
sess.run(init)

In [11]:
sess.run(linear_model, {x: range(1, 5)})

array([ 0.        ,  0.30000001,  0.60000002,  0.90000004], dtype=float32)

In [12]:
# desired values
y = tf.placeholder(tf.float32)
# loss funciton
squared_deltas = tf.square(linear_model - y)
loss = tf.reduce_sum(squared_deltas)
sess.run(loss, {x: range(1, 5), y: [0, -1, -2, -3]})

23.66

Variables can be reassigned.

In [13]:
fixW = tf.assign(W, [-1.])
fixb = tf.assign(b, [1.])
sess.run([fixW, fixb])
sess.run(loss, {x: range(1, 5), y: [0, -1, -2, -3]})

0.0

### tf.train API

TF provides **oprimizers** which minimize the loss
function.
The simplest is **gradient descent**.
TF can automatically produce derivatives given only a
desctiption of the model using `tf.gradients`.
Optimizers typically do this automatically.

In [14]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

In [15]:
x_train = [1., 2., 3., 4.]
y_train = [0., -1., -2., -3.]

sess.run(init) # reset variables
for i in range(1000):
    sess.run(train, {x: x_train, y: y_train})

curr_W, curr_b, curr_loss = sess.run(
    [W, b, loss],
    {x: x_train, y: y_train}
)
print('W: {}\nb: {}\nloss: {}'.format(curr_W, curr_b, curr_loss))

W: [-0.9999969]
b: [ 0.99999082]
loss: 5.699973826267524e-11


### tf.contrib.learn

`tf.contrib.learn` is a high-level TF library that
simplifies machine learning:

* running training loops
* running evaluation loops
* managing data sets
* managing feeding
* defines many common models

#### Basic Usage

In [16]:
import numpy as np

In [17]:
# declare list of features
features = [tf.contrib.layers.real_valued_column(
    'x',
    dimension=1
)]

# estimator is the front end to invoke training and
# evaluation
estimator = tf.contrib.learn.LinearRegressor(
    feature_columns=features
)

# helper method to read and set up dataset
x = np.array(x_train)
y = np.array(y_train)
input_fn = tf.contrib.learn.io.numpy_input_fn(
    {'x': x},
    y,
    batch_size=4,
    num_epochs=1000
)

# invoke training
estimator.fit(input_fn=input_fn, steps=1000)

# evaluate how well the model did
estimator.evaluate(input_fn=input_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_steps': None, '_num_worker_replicas': 0, '_model_dir': None, '_tf_random_seed': None, '_keep_checkpoint_every_n_hours': 10000, '_task_type': None, '_is_chief': True, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_keep_checkpoint_max': 5, '_save_summary_steps': 100, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f05c3d557f0>, '_evaluation_master': '', '_environment': 'local', '_task_id': 0, '_save_checkpoints_secs': 600, '_master': '', '_num_ps_replicas': 0}
Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:S

{'global_step': 1000, 'loss': 2.8872867e-08}

#### Custom Model

In [18]:
def model(features, labels, mode):
    W = tf.get_variable('W', [1], dtype=tf.float64)
    b = tf.get_variable('b', [1], dtype=tf.float64)
    y = W * features['x'] + b
    loss = tf.reduce_sum(tf.square(y - labels))
    global_step = tf.train.get_global_step()
    oprimizer = tf.train.GradientDescentOptimizer(0.01)
    train = tf.group(
        optimizer.minimize(loss),
        tf.assign_add(global_step, 1)
    )
    return tf.contrib.learn.ModelFnOps(
        mode=mode,
        predictions=y,
        loss=loss,
        train_op=train,
    )

estimator = tf.contrib.learn.Estimator(model_fn=model)
estimator.fit(input_fn=input_fn, steps=1000)
estimator.evaluate(input_fn=input_fn, steps=10)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_steps': None, '_num_worker_replicas': 0, '_model_dir': None, '_tf_random_seed': None, '_keep_checkpoint_every_n_hours': 10000, '_task_type': None, '_is_chief': True, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_keep_checkpoint_max': 5, '_save_summary_steps': 100, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f05f97196a0>, '_evaluation_master': '', '_environment': 'local', '_task_id': 0, '_save_checkpoints_secs': 600, '_master': '', '_num_ps_replicas': 0}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpn34ik9ti/model.ckpt.
INFO:tensorflow:step = 1, loss = 215.082694619
INFO:tensorflow:global_step/sec: 370.57
INFO:tensorflow:step = 101, loss = 0.194454121533 (0.272 sec)
INFO:tensorflow:global_step/sec: 393.316
INFO:tensorflow:step = 201, loss = 0.033620325925 (0.255 sec)
INFO:tensorflow:glo

{'global_step': 1000, 'loss': 7.1225123e-11}